# OXゲーム

## 問題定義

In [16]:
# Use English for comments
from dataclasses import dataclass
from typing import Iterable, Optional
from game_analyzer import State, Game, Solver


@dataclass
class OXState(State):
    # 0 = empty, 1 = O, 2 = X
    board: list[list[int]]

    def flip_board(self) -> 'OXState':
        # Flip 1 <-> 2, keep 0 as is
        # This treats the opponent's board as if it were my own turn
        new_board = [
            [2 if cell == 1 else 1 if cell == 2 else 0 for cell in row]
            for row in self.board
        ]
        return OXState(board=new_board)


class OXGame(Game):
    def __init__(self):
        # 3x3 board as default
        self.init_state = OXState(board=[
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0]
        ])

    def find_next_states(self, state: OXState) -> Iterable[OXState]:
        # Generate next states by placing "2" (my piece) on any empty cell (0)
        # Then flip the board to represent the opponent's turn as my turn perspective
        states = []
        size = len(state.board)
        for r in range(size):
            for c in range(size):
                if state.board[r][c] == 0:
                    new_board = [row[:] for row in state.board]  # copy
                    new_board[r][c] = 2
                    # After placing my piece, flip board => opponent's turn is seen as my turn
                    flipped = OXState(board=new_board).flip_board()
                    states.append(flipped)
        return states

    def find_mirror_states(self, state: OXState) -> Iterable[OXState]:
        # For a 3x3 board, we can generate rotations and flips as symmetric positions
        size = len(state.board)

        # Convert list[list[int]] -> tuple[tuple[int]]
        def to_tuple(bd: list[list[int]]) -> tuple[tuple[int]]:
            return tuple(tuple(row) for row in bd)

        # Convert tuple[tuple[int]] -> list[list[int]]
        def to_list(bd: tuple[tuple[int]]) -> list[list[int]]:
            return [list(row) for row in bd]

        def rotate_90(bd: list[list[int]]) -> list[list[int]]:
            # b'[r][c] = b[size-1-c][r]
            rotated = [[0]*size for _ in range(size)]
            for r in range(size):
                for c in range(size):
                    rotated[r][c] = bd[size - 1 - c][r]
            return rotated

        def flip_horizontal(bd: list[list[int]]) -> list[list[int]]:
            # flip each row horizontally
            flipped = [row[::-1] for row in bd]
            return flipped

        original = to_tuple(state.board)
        transformations = set()
        transformations.add(original)

        # Generate transformations
        current = to_list(original)
        for _ in range(3):
            current = rotate_90(current)
            transformations.add(to_tuple(current))

        flipped = flip_horizontal(to_list(original))
        transformations.add(to_tuple(flipped))

        # Rotate the flipped versions
        for _ in range(3):
            flipped = rotate_90(flipped)
            transformations.add(to_tuple(flipped))

        # Build OXState objects
        mirror_states = []
        for tbd in transformations:
            mirror_states.append(
                OXState(board=to_list(tbd))
            )
        return mirror_states

    def evaluate_state(self, state: OXState) -> Optional[int]:
        # For 3x3 board: 
        # return 1 if X(=2) wins, -1 if O(=1) wins, 0 if draw, else None
        bd = state.board
        size = len(bd)

        # Check rows
        for r in range(size):
            if bd[r][0] == bd[r][1] == bd[r][2] != 0:
                return 1 if bd[r][0] == 2 else -1

        # Check columns
        for c in range(size):
            if bd[0][c] == bd[1][c] == bd[2][c] != 0:
                return 1 if bd[0][c] == 2 else -1

        # Check diagonals
        if bd[0][0] == bd[1][1] == bd[2][2] != 0:
            return 1 if bd[0][0] == 2 else -1
        if bd[0][2] == bd[1][1] == bd[2][0] != 0:
            return 1 if bd[0][2] == 2 else -1

        # Check draw
        # If no empty cell (0) found => draw
        if all(bd[r][c] != 0 for r in range(size) for c in range(size)):
            return 0

        # Not finished yet
        return None


In [18]:
oxgame = OXGame()
solver = Solver()
result = solver.solve(oxgame)

In [19]:
result.state_to_params(oxgame.init_state)

(0, 9)

In [20]:
f"{result.sgg_time=}, {result.ra_time=}"

'result.sgg_time=0.06373977661132812, result.ra_time=0.0009500980377197266'